# Building a Computer

In this notebook we are going to design our own "computer" called Simpletron - a fairly basic computer that only understands its own "Simpletron Machine Language (SML)."

## Machine Language
Before we can write our computer, we have to understand what machine language is.
    
In the beginning, the only programming language was binary - 0's and 1's. Binary is incredibly difficult for humans to understand so the next step up is to program in decimal "operation codes" - op codes - that directly reference memory.

An SML instruction is a 2 byte *word* that consists of a one byte instruction and a one byte memory location.
    
In the short example below, we see the memory location containing the instruction in the left column, the instruction / memory location combo in the middle, and an English description of the operation in the right column.
```
Location   Number  Instruction
00         +1007   (Read variable A)
01         +1008   (Read variable B)
02         +2007   (Load variable A)
03         +3008   (Add variable B)
04         +2109   (Store result C)
05         +1109   (Write result C)
06         +4300   (Halt)
07         +0000   (Variable A)
08         +0000   (Variable B)
09         +0000   (Result C)
```

## Registers
Computers have special memory locations in the CPU called "registers" for holding and manipulating data. One special register is called the "**accumulator**."

The SML interpreter we are building will load data into the accumulator before it is used in calculations or some other way.

## Anatomy of a program
SML programs must be loaded into memory starting at location 00 and the Simpletron computer has a 100 word memory capacity (00 to 99). SML programs always begin executing at line 00.

Each memory location can contain an instruction, data, or be unused.

Instructions are composed of two parts: 
1. The opcode (operation to be performed)
2. The operand (address of memory location containing the word to be operated on)

|Operation Code|Meaning|
|--------------|-------|
|Input / Output operations||
|READ = 10|Read word from keyboard, store in memory|
|WRITE = 11|Write word to specific memory location| 
|Load and store operations||
|LOAD = 20|Load word from specific memory location|
|STORE = 21|Store word in specific memory location| 
|Arithmetic operations||
|ADD = 30|Add a word from a specific memory location to word in accumulator|
|SUBTRACT = 31|Subtract a word from a specific memory location from word in accumulator|
|DIVIDE = 32|Divide a word from a specific memory location into word in accumulator|
|MULTIPLY = 33|Multiply a word from a specific memory location with word in accumulator|
|Transfer of control||
|Branch = 40|Branch to specific memory location|
|BRANCHNEG = 41|Branch to memory location if accumulator is negative|
|BRANCHZERO = 42|Branch to specific memory location if accumulator is zero|
|HALT = 43|Halt - End program|

# Computer Simulator
The Simpletron computer is a software simulation of a primitive computer, but even so, you will be able to load, debug and run SML programs.

## Running the Simulation
The simulator should print a banner to inform and welcome the user: 
```
*** Welcome to Simpletron! ***

*** Please enter your program one instruction ***
*** (or data word) at a time. I will type the ***
*** location number and a question mark (?).  ***
*** You then type the word for that location. ***
*** Type the sentinel -99999 to stop entering  ***
*** your program                              ***
```


## Memory and Program Loading
The simulator's memory is a ** 100 element list. **

After printing the banner, the simulator will begin accepting program input by displaying the first memory location and a ?.
```
00 ?
```
The user will type program instructions line by line unti finished, then enter -99999 to end.
```
00 ? +1009
01 ? +1010
02 ? +2009
03 ? +3110
04 ? +4107
05 ? +1109
06 ? +4300
07 ? +0000
08 ? +0000
09 ? -99999

*** Program loading completed
*** Program execution begins
```

Program execution begins with line 00 and continues sequentially unless one of the BRANCH instructions are encountered.

** Variables **
* **accumulator** - the accumulator register
* **instruction_counter** - keeps track of the location in memory that contains the instruction to be performed
* **operation_code** - operation to be performed (left two digits of the instruction word)
* **operand** - memory location the current instruction operates on (right two digits)
* **instruction_register** - variable containing instruction to execute (Do not execute instructions directly from memory).


## Instruction Execution Cycle
Using instruction 00 in the example above,
**instruction_counter** tells us the location of the next instruction to load. 

### *Fetch* contents of the memory location:
```
instruction_register = memory[instruction_counter]
```
Extract the opcode and operand:
```
operation_code = instruction_register[0:2] # 10 - read
operand = instruction_register[2:4]        # 09 - into mem loc 09
```
Now Simpletron must examine the operation_code and determine:
* if it is a valid opcode
* what operation to perform

Ex:
```
read:   memory[ operand ] = input('? ')
load:   accumulator = memory[ operand ]
add :   accumulator += memory[ operand ]
branch:  
halt:   print('*** Simpletron Execution Terminated ***')
```

### Execute operand
Instruction word +1009 means "read from keyboard and put result in location 09."

Show a ? as a prompt for the read. When the user presses the 'Enter' key, place the value in 09.

**The instruction is complete at this point.**

### Prepare for next instruction
Increment instruction_counter:
```
instruction_counter += 1
```
**This completes the first instruction cycle.**

## Branching
Transfer of control (branching) means jumping foward or backward in the code depending on some condition. This can be accomplished by adjusting the value in the instruction_counter. For example, the "branch if accumulator zero" could be
```
if accumulator == 0:
   instruction_counter = operand
```
and the unconditional branch (40) could be
```
instruction_counter = operand
```